In [1]:
from utz import *
from njdot.crashes import Crashes
from njdot.data import cc2cn
from njdot.paths import CC2MC2MN

In [2]:
%%time
c = Crashes.load()
c

CPU times: user 27.8 s, sys: 19.8 s, total: 47.5 s
Wall time: 1min 5s


,year,cc,mc,case,cn,mn,pdc,pdn,station,tk,ti,pk,pi,severity,Intersection,Alcohol Involved,HazMat Involved,crash_type,tv,road,road_direction,Route,Route Suffix,sri,mp,Road System,Road Character,Road Surface Type,Surface Condition,Light Condition,Environmental Condition,Road Divided By,Temporary Traffic Control Zone,Distance To Cross Street,Unit Of Measurement,Direction From Cross Street,cross_street,Is Ramp,Ramp To/From Route Name,Ramp To/From Route Direction,Posted Speed,Posted Speed Cross Street,olat,olon,Cell Phone In Use Flag,Other Property Damage,Reporting Badge No.,dt,Road Horizontal Alignment,Road Grade,First Harmful Event
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,2001,1,1,#2001-17846,Atlantic,Absecon City,01,Absecon City,MUNICIPAL COMP?,0,0,0,0,p,B,False,False,03,2,CALDERON AVENUE,,,,,NaN,07,02,02,01,06,01,05,01,100,FE,N,RT 30,,,,25,,NaN,NaN,N,NEVER SAW V-1 MINOR DAMAGE - NO INJURIES REPOR...,830,2001-12-21 18:34:00,NaN,NaN,NaN
1,2001,1,1,01-00029,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,B,False,False,06,2,RITZ DRIVE,,,,,NaN,07,01,02,03,06,01,05,01,,,,,,,,25,,NaN,NaN,N,,836,2001-01-01 09:30:00,NaN,NaN,NaN
2,2001,1,1,01-004615,Atlantic,Absecon City,1,Absecon,,0,4,0,0,i,I,False,False,03,2,MORTON AVENUE,,,,,NaN,07,01,02,01,01,01,05,01,,AT,,NEW YORK AVENUE,,,,25,,NaN,NaN,N,,836,2001-04-10 14:44:00,NaN,NaN,NaN
3,2001,1,1,01-004880,Atlantic,Absecon City,1,Absecon,,0,1,0,0,i,B,False,False,01,2,RT 30,,30,,00000030__,51.10,02,01,02,01,01,01,05,01,,,,,,,,45,,NaN,NaN,N,WITNESS-PETRIA GIBSON AND ANGELO HERSH,886,2001-04-15 13:56:00,NaN,NaN,NaN
4,2001,1,1,01-004912,Atlantic,Absecon City,1,Absecon,,0,0,0,0,p,I,False,False,01,2,CR 651,,651,,,NaN,05,02,02,02,01,02,05,01,,AT,,CR 630,,,,35,,NaN,NaN,N,*BETH VEHICLES-RELIANCE INSURANCE CO.(215)864-...,836,2001-04-16 10:29:00,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6077298,2021,21,23,B150-2021-00117A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,11,1,WARREN COUNTY 620,N,620,,21000620__,0.00,05,,02,01,06,01,05,01,,AT,,ROUTE 519 / S BRIDGEVILLE RD,,,,50,50,40.80707,-75.07559,N,Damage to 10ft of Country road guardrail.,7837,2021-05-26 21:08:00,02,04,46
6077299,2021,21,23,B150-2021-00118A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,I,False,False,01,2,MANUNKA CHUNK RD / UPPER SEREPTA RD **,W,,,21231034__,1.73,07,,02,01,01,01,05,01,,AT,,MASSENATTS RD,,,,40,50,40.84466,-75.05549,N,,8610,2021-05-25 07:26:00,01,04,26
6077300,2021,21,23,B150-2021-00128A,Warren,White Twp,02,New Jersey State Police,WASHINGTON - SQ,0,0,0,0,p,B,False,False,12,1,ROUTE 519,S,519,,00000519__,44.00,05,,02,02,05,04,05,01,,,,,,,,50,,40.80564,-75.07627,N,,8514,2021-06-03 20:17:00,03,04,24


Helper that returns:
1. unique sets of values found in `cols`
2. rows from 1. that match all columns but the last

In [3]:
def ambiguous_mappings(df, cols):
    uniqs = (
        df
        .reset_index()
        [cols]
        .drop_duplicates()
        .sort_values(cols)
    )
    hist = uniqs.value_counts(cols[:-1], sort=False)
    conflicts = hist[hist > 1]
    conflicts = uniqs.set_index(cols[:-1]).loc[conflicts.index].reset_index()
    return uniqs, conflicts

## County Code ⟹ County Name

In [4]:
%%time
cn_cols = [ 'cc', 'cn' ]
_, county_name_conflicts = ambiguous_mappings(c, cn_cols)
assert len(county_name_conflicts) == 0
c.reset_index()[cn_cols].value_counts(sort=False).rename('# Crashes')

CPU times: user 2.53 s, sys: 4.32 s, total: 6.85 s
Wall time: 8.81 s


cc  cn        
1   Atlantic      192238
2   Bergen        656350
3   Burlington    264893
4   Camden        324947
5   Cape May       65356
6   Cumberland     98515
7   Essex         610767
8   Gloucester    155854
9   Hudson        424523
10  Hunterdon      83811
11  Mercer        271333
12  Middlesex     606975
13  Monmouth      416092
14  Morris        325609
15  Ocean         340102
16  Passaic       390426
17  Salem          36922
18  Somerset      237074
19  Sussex         78639
20  Union         425445
21  Warren         71432
Name: # Crashes, dtype: int64

## { County Code, Muni Code } ⟹ Muni Name conflicts

In [5]:
%%time
mn_keys = [ 'cc', 'mc' ]
mn_val = 'mn'
mn_cols = mn_keys + [ mn_val ]
_, muni_name_conflicts = ambiguous_mappings(c, mn_cols)
muni_name_conflicts.set_index(mn_cols)

CPU times: user 1.3 s, sys: 2.14 s, total: 3.44 s
Wall time: 4.31 s


Empty DataFrame
Columns: []
Index: [(14, 30, Long Hill Twp), (14, 30, Passaic Twp), (15, 7, Dover Twp), (15, 7, Toms River Twp), (16, 16, West Paterson Boro), (16, 16, Woodland Park Boro)]

## { County Code, Muni Code, Year } ⟹ Muni Name

In [6]:
%%time
mny_keys = [ 'cc', 'mc', 'year' ]
mny_cols = mny_keys + [ mn_val ]
mny_uniqs, mny_conflicts = ambiguous_mappings(c, mny_cols)
assert len(mny_conflicts) == 0
mny_uniqs.set_index(mn_cols + ['year'])

CPU times: user 1.32 s, sys: 2.09 s, total: 3.41 s
Wall time: 4.27 s


Empty DataFrame
Columns: []
Index: [(1, 1, Absecon City, 2001), (1, 1, Absecon City, 2002), (1, 1, Absecon City, 2003), (1, 1, Absecon City, 2004), (1, 1, Absecon City, 2005), (1, 1, Absecon City, 2006), (1, 1, Absecon City, 2007), (1, 1, Absecon City, 2008), (1, 1, Absecon City, 2009), (1, 1, Absecon City, 2010), (1, 1, Absecon City, 2011), (1, 1, Absecon City, 2012), (1, 1, Absecon City, 2013), (1, 1, Absecon City, 2014), (1, 1, Absecon City, 2015), (1, 1, Absecon City, 2016), (1, 1, Absecon City, 2017), (1, 1, Absecon City, 2018), (1, 1, Absecon City, 2019), (1, 1, Absecon City, 2020), (1, 1, Absecon City, 2021), (1, 2, Atlantic City, 2001), (1, 2, Atlantic City, 2002), (1, 2, Atlantic City, 2003), (1, 2, Atlantic City, 2004), (1, 2, Atlantic City, 2005), (1, 2, Atlantic City, 2006), (1, 2, Atlantic City, 2007), (1, 2, Atlantic City, 2008), (1, 2, Atlantic City, 2009), (1, 2, Atlantic City, 2010), (1, 2, Atlantic City, 2011), (1, 2, Atlantic City, 2012), (1, 2, Atlantic City, 2013), (1, 2, Atlantic City, 2014), (1, 2, Atlantic City, 2015), (1, 2, Atlantic City, 2016), (1, 2, Atlantic City, 2017), (1, 2, Atlantic City, 2018), (1, 2, Atlantic City, 2019), (1, 2, Atlantic City, 2020), (1, 2, Atlantic City, 2021), (1, 3, Brigantine City, 2001), (1, 3, Brigantine City, 2002), (1, 3, Brigantine City, 2003), (1, 3, Brigantine City, 2004), (1, 3, Brigantine City, 2005), (1, 3, Brigantine City, 2006), (1, 3, Brigantine City, 2007), (1, 3, Brigantine City, 2008), (1, 3, Brigantine City, 2009), (1, 3, Brigantine City, 2010), (1, 3, Brigantine City, 2011), (1, 3, Brigantine City, 2012), (1, 3, Brigantine City, 2013), (1, 3, Brigantine City, 2014), (1, 3, Brigantine City, 2015), (1, 3, Brigantine City, 2016), (1, 3, Brigantine City, 2017), (1, 3, Brigantine City, 2018), (1, 3, Brigantine City, 2019), (1, 3, Brigantine City, 2020), (1, 3, Brigantine City, 2021), (1, 4, Buena Boro, 2001), (1, 4, Buena Boro, 2002), (1, 4, Buena Boro, 2003), (1, 4, Buena Boro, 2004), (1, 4, Buena Boro, 2005), (1, 4, Buena Boro, 2006), (1, 4, Buena Boro, 2007), (1, 4, Buena Boro, 2008), (1, 4, Buena Boro, 2009), (1, 4, Buena Boro, 2010), (1, 4, Buena Boro, 2011), (1, 4, Buena Boro, 2012), (1, 4, Buena Boro, 2013), (1, 4, Buena Boro, 2014), (1, 4, Buena Boro, 2015), (1, 4, Buena Boro, 2016), (1, 4, Buena Boro, 2017), (1, 4, Buena Boro, 2018), (1, 4, Buena Boro, 2019), (1, 4, Buena Boro, 2020), (1, 4, Buena Boro, 2021), (1, 5, Buena Vista Twp, 2001), (1, 5, Buena Vista Twp, 2002), (1, 5, Buena Vista Twp, 2003), (1, 5, Buena Vista Twp, 2004), (1, 5, Buena Vista Twp, 2005), (1, 5, Buena Vista Twp, 2006), (1, 5, Buena Vista Twp, 2007), (1, 5, Buena Vista Twp, 2008), (1, 5, Buena Vista Twp, 2009), (1, 5, Buena Vista Twp, 2010), (1, 5, Buena Vista Twp, 2011), (1, 5, Buena Vista Twp, 2012), (1, 5, Buena Vista Twp, 2013), (1, 5, Buena Vista Twp, 2014), (1, 5, Buena Vista Twp, 2015), (1, 5, Buena Vista Twp, 2016), ...]

[11747 rows x 0 columns]

In [7]:
cc2mc2mn = (
    mny_uniqs
    [mny_uniqs.year == 2021]
    .set_index('cc')
    [['mc', 'mn']]
    .groupby(lambda x:x)
    .apply(lambda df: df.set_index('mc')['mn'].to_dict())
    .to_dict()
)
cc2mc2mn = {
    cc: dict(cn=cc2cn[cc], mc2mn=mc2mn)
    for cc, mc2mn in cc2mc2mn.items()
}
cc2mc2mn

{1: {'cn': 'Atlantic',
  'mc2mn': {1: 'Absecon City',
   2: 'Atlantic City',
   3: 'Brigantine City',
   4: 'Buena Boro',
   5: 'Buena Vista Twp',
   6: 'Corbin City',
   7: 'Egg Harbor City',
   8: 'Egg Harbor Twp',
   9: 'Estell Manor City',
   10: 'Folsom Boro',
   11: 'Galloway Twp',
   12: 'Hamilton Twp',
   13: 'Hammonton Town',
   14: 'Linwood City',
   15: 'Longport Boro',
   16: 'Margate City',
   17: 'Mullica Twp',
   18: 'Northfield City',
   19: 'Pleasantville City',
   20: 'Port Republic City',
   21: 'Somers Point City',
   22: 'Ventnor City',
   23: 'Weymouth Twp'}},
 2: {'cn': 'Bergen',
  'mc2mn': {1: 'Allendale Boro',
   2: 'Alpine Boro',
   3: 'Bergenfield Boro',
   4: 'Bogota Boro',
   5: 'Carlstadt Boro',
   6: 'Cliffside Park Boro',
   7: 'Closter Boro',
   9: 'Demarest Boro',
   10: 'Dumont Boro',
   11: 'Elmwood Park Boro',
   12: 'East Rutherford Boro',
   13: 'Edgewater Boro',
   14: 'Emerson Boro',
   15: 'Englewood City',
   16: 'Englewood Cliffs Boro',
   17

In [8]:
with open(CC2MC2MN, 'w') as f:
    json.dump(cc2mc2mn, f, indent=2)

## Muni Name transitions

In [9]:
%%time
mnys = (
    mny_uniqs
    .merge(muni_name_conflicts, on=mn_cols)
    .set_index(mny_keys)
    .sort_index()
    [mn_val]
)
mnys

CPU times: user 5.97 ms, sys: 2.1 ms, total: 8.07 ms
Wall time: 9.3 ms


cc  mc  year
14  30  2001           Passaic Twp
        2002           Passaic Twp
        2003           Passaic Twp
        2004           Passaic Twp
        2005           Passaic Twp
        2006           Passaic Twp
        2007           Passaic Twp
        2008           Passaic Twp
        2009           Passaic Twp
        2010           Passaic Twp
        2011           Passaic Twp
        2012           Passaic Twp
        2013         Long Hill Twp
        2014         Long Hill Twp
        2015         Long Hill Twp
        2016         Long Hill Twp
        2017           Passaic Twp
        2018           Passaic Twp
        2019           Passaic Twp
        2021           Passaic Twp
15  7   2001             Dover Twp
        2002             Dover Twp
        2003             Dover Twp
        2004             Dover Twp
        2005             Dover Twp
        2006             Dover Twp
        2007             Dover Twp
        2008        Toms River Twp
       

In [10]:
mnys[mnys != mnys.shift(1)]

cc  mc  year
14  30  2001           Passaic Twp
        2013         Long Hill Twp
        2017           Passaic Twp
15  7   2001             Dover Twp
        2008        Toms River Twp
16  16  2001    West Paterson Boro
        2008    Woodland Park Boro
Name: mn, dtype: object

## Verify Muni Codes auto-increment within each County

In [13]:
%%time
codes_hist = c.reset_index()[['cc', 'mc']].astype(int).value_counts(sort=False)
codes_hist

CPU times: user 1.11 s, sys: 2.23 s, total: 3.34 s
Wall time: 4.45 s


cc  mc
1   1      7462
    2     43797
    3      2958
    4      2057
    5      4150
          ...  
21  19    10066
    20     5204
    21     2757
    22     3853
    23     2918
Name: count, Length: 568, dtype: int64

In [14]:
def fsck_codes(s):
    l = s.tolist()
    return l == list(range(1, len(l) + 1)), f"{l}"

assert (
    codes_hist
    .reset_index()
    .groupby('cc')
    ['mc']
    .apply(fsck_codes)
).all()